In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import torch
torch.manual_seed(100)

#Data Loading

In [ ]:
dataset1 = pd.read_csv('/content/drive/MyDrive/CSE499B/CSE499B_merged_data/ERPdata.csv')
dataset = dataset1.drop(columns=['time_ms'])
dataset.head(5)

,subject,condition,Fz,FCz,Cz,FC3,FC4,C3,C4,CP3,CP4
0,1,1,5.533701,5.726507,5.469535,5.386723,4.588875,6.560092,4.542811,5.397492,5.103695
1,1,1,5.651489,5.837326,5.773131,5.627975,4.822217,6.739976,4.811770,5.541357,5.379273
2,1,1,5.717580,5.932924,5.948466,5.826460,4.979647,7.026199,5.053779,5.634972,5.600504
3,1,1,5.703267,5.968103,5.851512,5.812192,4.992899,6.940671,5.106650,5.543577,5.589775
4,1,1,5.571578,5.917541,5.812808,5.744715,4.963338,6.726491,5.158073,5.454069,5.614092


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print('Total amount of files in ERP directory: ' + str(len(os.listdir('/content/drive/MyDrive/CSE499B/CSE499B_merged_data/ERP_81'))))


Total amount of files in ERP directory: 81


In [ ]:
import numpy as np
import pandas as pd 
import os
import random
from tqdm import tqdm
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from scipy.stats import mannwhitneyu

#print('Total amount of files in SMNI_CMI_TRAIN directory: ' + str(len(os.listdir('/kaggle/input/Alcoholics/SMNI_CMI_TRAIN/Train'))))

filenames_list = os.listdir('/content/drive/MyDrive/CSE499B/CSE499B_merged_data/ERP_81/') ## list of file names in the directory
#EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file



for file_name in tqdm(filenames_list):
    temp_df = pd.read_csv('/content/drive/MyDrive/CSE499B/CSE499B_merged_data/ERP_81/' + file_name) ## read from the file to df
    #EEG_data = EEG_data.append(temp_df) ## add the file data to the main df


100%|██████████| 81/81 [00:01<00:00, 46.41it/s]


In [ ]:
dataset.shape

(746496, 11)

In [ ]:
!pip install eeglib

In [ ]:
!pip install librosa

In [ ]:
!pip install antropy

#Segments


1.   Segment 1

* Mean
* variance
* zero-crossing rate
* and approximate entropy
>(4 features × 9 channels)=36
	

2.   Segment 2

  Average PSD in 
* delta (1-4 HZ)
* theta (4–8 Hz)
* alpha (8–12 Hz)
* beta (12–30 Hz)
* gamma (30–45 HZ)

> (5 power × 9 channels)=45



                                                      **Not Done**

--------------------------------------------------------------------------------
3. Segment 3

  Difference of average PSD 
* Theta
* Alpha 
* Beta
* Gamma for 4 EEG channel pairs between right and left scalp 
>(4 power differences × 4 channel pairs)=16

4. Segment 4 

  Average values of 
* squared amplitude 
* instantaneous frequency
 
 delta (1–4 Hz), theta (4–8 Hz), alpha (8–12 Hz), beta (12–30 Hz), and gamma   ( 30–45 Hz) bands for all EEG channels 
 >(2 features × 5 bands × 9 channels) 



#Feature Vector

In [ ]:
from sys import ps1
import eeglib
import librosa
import antropy as ant
from statistics import variance
final_vector=np.array([])
for file_name in tqdm(filenames_list):
    
    temp_df = pd.read_csv('/content/drive/MyDrive/CSE499B/CSE499B_merged_data/ERP_81/' + file_name) ## read from the file to df 
    helper = eeglib.helpers.CSVHelper("/content/drive/MyDrive/CSE499B/CSE499B_merged_data/ERP_81/" + file_name, selectedSignals=['Fz','FCz','Cz','FC3','FC4','C3','C4','CP3','CP4'], normalize=True ,sampleRate=100)
    wrapper = eeglib.wrapper.Wrapper(helper)

                                                                                                              #####Segemnt-01#####
    # Mean
    f1c1=temp_df['Fz'].mean(axis = 0)
    f1c2=temp_df['FCz'].mean(axis = 0)
    f1c3=temp_df['Cz'].mean(axis = 0)
    f1c4=temp_df['FC3'].mean(axis = 0)
    f1c5=temp_df['FC4'].mean(axis = 0)
    f1c6=temp_df['C3'].mean(axis = 0)
    f1c7=temp_df['C4'].mean(axis = 0)
    f1c8=temp_df['CP3'].mean(axis = 0)
    f1c9=temp_df['CP4'].mean(axis = 0)

    # Variance
    f2c1=variance(temp_df['Fz'])
    f2c2=variance(temp_df['FCz'])
    f2c3=variance(temp_df['Cz'])
    f2c4=variance(temp_df['FC3'])
    f2c5=variance(temp_df['FC4'])
    f2c6=variance(temp_df['C3'])
    f2c7=variance(temp_df['C4'])
    f2c8=variance(temp_df['CP3'])
    f2c9=variance(temp_df['CP4'])

    # Zero-crossing rate
    x1=np.array(temp_df['Fz'])
    f3c1=librosa.zero_crossings(x1)
    f3c1=sum(f3c1)
    x2=np.array(temp_df['FCz'])
    f3c2=librosa.zero_crossings(x2)
    f3c2=sum(f3c2)
    x3=np.array(temp_df['Cz'])
    f3c3=librosa.zero_crossings(x3)
    f3c3=sum(f3c3)
    x4=np.array(temp_df['FC3'])
    f3c4=librosa.zero_crossings(x4)
    f3c4=sum(f3c4)
    x5=np.array(temp_df['FC4'])
    f3c5=librosa.zero_crossings(x5)
    f3c5=sum(f3c5)
    x6=np.array(temp_df['C3'])
    f3c6=librosa.zero_crossings(x6)
    f3c6=sum(f3c6)
    x7=np.array(temp_df['C4'])
    f3c7=librosa.zero_crossings(x7)
    f3c7=sum(f3c7)
    x8=np.array(temp_df['CP3'])
    f3c8=librosa.zero_crossings(x8)
    f3c8=sum(f3c8)
    x9=np.array(temp_df['CP4'])
    f3c9=librosa.zero_crossings(x9)
    f3c9=sum(f3c9)

    

    # approximate entropy
    f4c1=ant.app_entropy(x1)
    f4c2=ant.app_entropy(x2)
    f4c3=ant.app_entropy(x3)
    f4c4=ant.app_entropy(x4)
    f4c5=ant.app_entropy(x5)
    f4c6=ant.app_entropy(x6)
    f4c7=ant.app_entropy(x7)
    f4c8=ant.app_entropy(x8)
    f4c9=ant.app_entropy(x9)





   

                                                                                                              #####Segemnt-02#####

    wrapper.addFeature.bandPower(bands={'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma':(30,45)})# bandpower which extracts five bands in our case, starts from delta, theta, alpha and beta.
    wrapper.addFeature.DTW()# Dynamic Time Warping
    wrapper.addFeature.hjorthActivity()#Time domain: Hjorth Parameters
    wrapper.addFeature.HFD() #Higuchi Fractal Dimension
    wrapper.addFeature.PFD()#Petrosian Fractal Dimension 
    wrapper.addFeature.LZC()#Lempel–Ziv Complexity
    data_sch = wrapper.getAllFeatures()


    #HjorthActivity()
    f6c1 = data_sch['hjorthActivity(){}_0'].mean( axis =0)
    f6c2 = data_sch['hjorthActivity(){}_1'].mean( axis =0)
    f6c3 = data_sch['hjorthActivity(){}_2'].mean( axis =0)
    f6c4 = data_sch['hjorthActivity(){}_3'].mean( axis =0)
    f6c5 = data_sch['hjorthActivity(){}_4'].mean( axis =0)
    f6c6 = data_sch['hjorthActivity(){}_5'].mean( axis =0)
    f6c7 = data_sch['hjorthActivity(){}_6'].mean( axis =0)
    f6c8 = data_sch['hjorthActivity(){}_7'].mean( axis =0)
    f6c9 = data_sch['hjorthActivity(){}_8'].mean( axis =0)


    #HFD
    f7c1 = data_sch['HFD(){}_0'].mean( axis =0)
    f7c2 = data_sch['HFD(){}_1'].mean( axis =0)
    f7c3 = data_sch['HFD(){}_2'].mean( axis =0)
    f7c4 = data_sch['HFD(){}_3'].mean( axis =0)
    f7c5 = data_sch['HFD(){}_4'].mean( axis =0)
    f7c6 = data_sch['HFD(){}_5'].mean( axis =0)
    f7c7 = data_sch['HFD(){}_6'].mean( axis =0)
    f7c8 = data_sch['HFD(){}_7'].mean( axis =0)
    f7c9 = data_sch['HFD(){}_8'].mean( axis =0)


    #PFD
    f8c1 = data_sch['PFD(){}_0'].mean( axis =0)
    f8c2 = data_sch['PFD(){}_1'].mean( axis =0)
    f8c3 = data_sch['PFD(){}_2'].mean( axis =0)
    f8c4 = data_sch['PFD(){}_3'].mean( axis =0)
    f8c5 = data_sch['PFD(){}_4'].mean( axis =0)
    f8c6 = data_sch['PFD(){}_5'].mean( axis =0)
    f8c7 = data_sch['PFD(){}_6'].mean( axis =0)
    f8c8 = data_sch['PFD(){}_7'].mean( axis =0)
    f8c9 = data_sch['PFD(){}_8'].mean( axis =0)   


    #LZC()
    f9c1 = data_sch['LZC(){}_0'].mean( axis =0)
    f9c2 = data_sch['LZC(){}_1'].mean( axis =0)
    f9c3 = data_sch['LZC(){}_2'].mean( axis =0)
    f9c4 = data_sch['LZC(){}_3'].mean( axis =0)
    f9c5 = data_sch['LZC(){}_4'].mean( axis =0)
    f9c6 = data_sch['LZC(){}_5'].mean( axis =0)
    f9c7 = data_sch['LZC(){}_6'].mean( axis =0)
    f9c8 = data_sch['LZC(){}_7'].mean( axis =0)
    f9c9 = data_sch['LZC(){}_8'].mean( axis =0)  
                                                                                                             #####Segemnt-03#####
    
    #delta
    p1c1 = data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_0_delta"].mean(axis = 0)
    p1c2 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_1_delta"].mean(axis = 0)
    p1c3 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_2_delta"].mean(axis = 0)
    p1c4 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_3_delta"].mean(axis = 0)
    p1c5 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_4_delta"].mean(axis = 0)
    p1c6 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_5_delta"].mean(axis = 0)
    p1c7 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_6_delta"].mean(axis = 0)
    p1c8 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_7_delta"].mean(axis = 0)
    p1c9 =data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_8_delta"].mean(axis = 0)

    #theta
    p2c1=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_0_theta"].mean(axis = 0)
    p2c2=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_1_theta"].mean(axis = 0)
    p2c3=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_2_theta"].mean(axis = 0)
    p2c4=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_3_theta"].mean(axis = 0)
    p2c5=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_4_theta"].mean(axis = 0)
    p2c6=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_5_theta"].mean(axis = 0)
    p2c7=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_6_theta"].mean(axis = 0)
    p2c8=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_7_theta"].mean(axis = 0)
    p2c9=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_8_theta"].mean(axis = 0)

    #alpha
    p3c1=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_0_alpha"].mean(axis = 0)
    p3c2=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_1_alpha"].mean(axis = 0)
    p3c3=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_2_alpha"].mean(axis = 0)
    p3c4=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_3_alpha"].mean(axis = 0)
    p3c5=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_4_alpha"].mean(axis = 0)
    p3c6=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_5_alpha"].mean(axis = 0)
    p3c7=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_6_alpha"].mean(axis = 0)
    p3c8=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_7_alpha"].mean(axis = 0)
    p3c9=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_8_alpha"].mean(axis = 0)

    #beta
    p4c1=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_0_beta"].mean(axis = 0)
    p4c2=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_1_beta"].mean(axis = 0)
    p4c3=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_2_beta"].mean(axis = 0)
    p4c4=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_3_beta"].mean(axis = 0)
    p4c5=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_4_beta"].mean(axis = 0)
    p4c6=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_5_beta"].mean(axis = 0)
    p4c7=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_6_beta"].mean(axis = 0)
    p4c8=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_7_beta"].mean(axis = 0)
    p4c9=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_8_beta"].mean(axis = 0)
    
    #gamma
    p5c1=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_0_gamma"].mean(axis = 0)
    p5c2=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_1_gamma"].mean(axis = 0)
    p5c3=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_2_gamma"].mean(axis = 0)
    p5c4=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_3_gamma"].mean(axis = 0)
    p5c5=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_4_gamma"].mean(axis = 0)
    p5c6=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_5_gamma"].mean(axis = 0)
    p5c7=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_6_gamma"].mean(axis = 0)
    p5c8=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_7_gamma"].mean(axis = 0)
    p5c9=data_sch["bandPower(){'bands': {'alpha': (8, 12), 'beta': (12, 30), 'delta': (1, 4), 'theta': (4, 7), 'gamma': (30, 45)}}_8_gamma"].mean(axis = 0)

   



    feature=np.array([f1c1,f1c2,f1c3,f1c4,f1c5,f1c6,f1c7,f1c8,f1c9,f2c1,f2c2,f2c3,f2c4,f2c5,f2c6,f2c7,f2c8,f2c9,f3c1,f3c2,f3c3,f3c4,f3c5,f3c6,f3c7,f3c8,f3c9,f4c1,f4c2,f4c3,f4c4,f4c5,f4c6,f4c7,f4c8,f4c9,
                                             f6c1,f6c2,f6c3,f6c4,f6c5,f6c6,f6c7,f6c8,f6c9,f7c1,f7c2,f7c3,f7c4,f7c5,f7c6,f7c7,f7c8,f7c9,
                       f8c1,f8c2,f8c3,f8c4,f8c5,f8c6,f8c7,f8c8,f8c9,f9c1,f9c2,f9c3,f9c4,f9c5,f9c6,f9c7,f9c8,f9c9,
                       p1c1,p1c2,p1c3,p1c4,p1c5,p1c6,p1c7,p1c8,p1c9,p2c1,p2c2,p2c3,p2c4,p2c5,p2c6,p2c7,p2c8,p2c9,p3c1,p3c2,p3c3,p3c4,p3c5,p3c6,p3c7,p3c8,p3c9,p4c1,p4c2,p4c3,p4c4,p4c5,p4c6,p4c7,p4c8,p4c9,p5c1,p5c2,p5c3,p5c4,p5c5,p5c6,p5c7,p5c8,p5c9])
    final_vector = np.concatenate((final_vector, feature),)
    






100%|██████████| 81/81 [04:30<00:00,  3.34s/it]


In [ ]:
final_vector.shape

(9477,)

In [ ]:
final_vector=np.resize(final_vector,(81,117))
print(final_vector)


[[ 2.26031828  2.34951556  2.28510235 ...  3.40232997  3.17472242
   3.61689134]
 [ 0.32832976  0.3809611   0.4020188  ...  2.84332063  2.85571907
   3.36296418]
 [ 1.31923833  1.64993254  1.76459014 ...  1.88670699  1.61381843
   1.69607769]
 ...
 [-0.53074212 -0.25055261  0.71018101 ...  2.95850348  2.71479403
   2.93707523]
 [ 1.25191515  2.04121865  2.5663601  ...  2.40911251  1.77712402
   2.24592311]
 [-0.515761   -0.40778959 -0.43515915 ...  3.46170022  2.62443648
   3.16114898]]


In [ ]:
final_vector.shape

(81, 117)

In [ ]:
import pandas as pd
pd=pd.DataFrame(final_vector,columns=['f1c1','f1c2','f1c3','f1c4','f1c5','f1c6','f1c7','f1c8','f1c9','f2c1','f2c2','f2c3','f2c4','f2c5','f2c6','f2c7','f2c8','f2c9','f3c1','f3c2','f3c3','f3c4','f3c5','f3c6','f3c7','f3c8','f3c9','f4c1','f4c2','f4c3','f4c4','f4c5','f4c6','f4c7','f4c8','f4c9',
                         'f6c1','f6c2','f6c3','f6c4','f6c5','f6c6','f6c7','f6c8','f6c9','f7c1','f7c2','f7c3','f7c4','f7c5','f7c6','f7c7','f7c8','f7c9', 
                          'f8c1','f8c2','f8c3','f8c4','f8c5','f8c6','f8c7','f8c8','f8c9','f9c1','f9c2','f9c3','f9c4','f9c5','f9c6','f9c7','f9c8','f9c9',            
                       'p1c1','p1c2','p1c3','p1c4','p1c5','p1c6','p1c7','p1c8','p1c9','p2c1','p2c2','p2c3','p2c4','p2c5','p2c6','p2c7','p2c8','p2c9','p3c1','p3c2','p3c3','p3c4','p3c5','p3c6','p3c7','p3c8','p3c9','p4c1','p4c2','p4c3','p4c4','p4c5','p4c6','p4c7','p4c8','p4c9','p5c1','p5c2','p5c3','p5c4','p5c5','p5c6','p5c7','p5c8','p5c9'])

In [ ]:
pd

,f1c1,f1c2,f1c3,f1c4,f1c5,f1c6,f1c7,f1c8,f1c9,f2c1,...,p4c9,p5c1,p5c2,p5c3,p5c4,p5c5,p5c6,p5c7,p5c8,p5c9
0,2.260318,2.349516,2.285102,2.371036,1.723475,2.354274,1.938889,2.088486,1.822621,4.532200,...,15.575810,2.847430,2.468363,2.475560,2.852187,3.900093,2.878861,3.402330,3.174722,3.616891
1,0.328330,0.380961,0.402019,0.302958,0.721228,0.672094,0.279890,0.338119,0.285035,1.727734,...,12.206533,2.638754,2.326181,2.539087,2.771569,3.194668,2.511841,2.843321,2.855719,3.362964
2,1.319238,1.649933,1.764590,1.534097,1.306506,2.514999,1.442240,2.010884,1.819087,6.019772,...,7.143419,1.797214,1.589979,1.508731,1.750036,1.837351,1.716106,1.886707,1.613818,1.696078
3,0.881155,0.549112,-0.009987,0.481874,0.616842,0.255339,0.236800,-0.273485,0.047532,5.899954,...,11.188705,2.795483,2.967400,2.478721,3.613074,4.194302,3.455086,3.332238,3.230821,3.102742
4,2.480299,2.159354,2.323685,1.121081,2.003411,1.788438,2.149194,1.761871,1.609207,18.623084,...,12.122419,1.722872,1.842653,1.870039,2.220646,2.627883,2.439646,2.240589,2.656710,2.838000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,-1.157693,-0.473760,0.066035,0.051544,-0.085585,0.665702,0.074651,0.655098,-0.058508,2.935724,...,13.219561,3.267912,3.792989,3.093183,4.853135,4.141656,4.564491,3.630951,3.682967,3.549432
77,0.525635,0.871797,1.624414,1.139258,0.289800,1.603670,1.494863,1.057946,1.627259,3.165561,...,8.427416,2.575738,2.144823,1.843000,2.608192,2.423512,2.448702,2.245097,2.530396,2.153556
78,-0.530742,-0.250553,0.710181,-0.213728,0.069126,0.500686,0.414643,0.783273,0.578769,2.071274,...,10.054090,2.739644,2.923904,3.020601,2.674084,3.190042,3.062861,2.958503,2.714794,2.937075
79,1.251915,2.041219,2.566360,1.948045,1.074758,2.529576,1.409276,1.946375,1.088037,4.455269,...,8.919213,2.493930,1.827174,1.595566,1.969464,2.897315,1.634551,2.409113,1.777124,2.245923


In [ ]:
path1= '/content/drive/MyDrive/CSE499B/'
pd.to_csv(path1+'Feature_vector_108.csv', index=None)

In [ ]:
final_vector.size

9477

In [ ]:
final_vector=np.resize(final_vector,(9477,1))